In [1]:
import torch
import numpy as np

from models.wta_cnp import WTA_CNP

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [2]:
model = WTA_CNP().to(device)
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())
# model

WTA_CNP(
  (encoder): Sequential(
    (0): Linear(in_features=2, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
  )
  (decoders): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=257, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=2, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=257, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=2, bias=True)
    )
    (2): Sequential(
      (0): Linear(in_features=257, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=2, bias=True)
    )
    (3): Sequential(
      (0): Linear(in_features=257, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=2, bias=True)
    )
  )
  (gate): Sequential(
    (0): L

In [4]:
o, t = torch.rand(1, 1, 2, device=device), torch.rand(1, 1, 1, device=device)
p = model(x, x[0])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x200 and 2x256)